**WEEK - 3 PART ONE**

*Importing Libraries*

In [24]:
import numpy as np
import pandas as pd
import urllib

*Scraping Information*

In [25]:
from urllib.request import Request, urlopen

req = Request('http://www.wikizero.biz/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N', headers={'User-Agent': 'Mozilla/5.0'})
content = urlopen(req).read()

In [26]:
decoded_content = content.decode("UTF-8")

In [27]:
postcode_table = decoded_content[decoded_content.find("<table"):decoded_content.find("</table>")+8]

In [28]:
postcode_df = pd.read_html(postcode_table, header = 0)[0]

*Removing "Not Assigned" Rows*

In [29]:
postcode_df= postcode_df[postcode_df.Borough != "Not assigned"]
postcode_df.Neighbourhood[postcode_df.Neighbourhood == "Not assigned"] =postcode_df.Borough[postcode_df.Neighbourhood == "Not assigned"]
postcode_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


*Writing Function for Series of Hoods*

In [30]:
def concat_nhood(x):
    cadena = ""
    for i in range(len(x)-1):
        cadena = cadena + x.iloc[i] + ", "
    cadena += x.iloc[-1]
    return cadena

def select_Borough(x):
    ref = x.iloc[0]
    for i in range(1, len(x)):
        if ref != x.iloc[i]:
            for i in x:
                print(x)
            raise Exception("Postcode comprises two Boroughs")
    return ref

*Group Data by "Postcode"*

In [31]:
df_group = postcode_df.groupby(["Postcode"]).agg({"Borough": lambda x: select_Borough(x),
                                 "Neighbourhood": lambda x: concat_nhood(x)})
df_group.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [32]:
df_group.shape

(103, 2)

In [34]:
df_group.to_csv("toronto_hoods.csv", encoding='utf-8')